In [1]:
import torch
from torch import nn

In [ ]:
class RRDBBlock(nn.Module):
    def __init__(self, channels, growth_channels = 32) -> None:
        super(RRDBBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, growth_channels, 3, 1, 1)
        self.conv2 = nn.Conv2d(growth_channels, channels + growth_channels, 3, 1, 1)
        self.conv3 = nn.Conv2d(channels + growth_channels, channels + 2 * growth_channels, 3, 1, 1)
        self.conv4 = nn.Conv2d(channels + 2 * growth_channels, channels + 3 * growth_channels, 3, 1, 1)
        self.conv5 = nn.Conv2d(channels + 3 * growth_channels, channels + 4 * growth_channels, 3, 1, 1)
        self.lrelu = nn.LeakyReLU(negative_slope=0.2, inplace=True)
    def forward(self, x) -> torch.Tensor:
        x1 = self.lrelu(self.conv1(x))
        x2 = self.lrelu(self.conv2(torch.cat([x, x1], 1)))
        x3 = self.lrelu(self.conv3(torch.cat([x, x1, x2], 1)))
        x4 = self.lrelu(self.conv4(torch.cat([x, x1, x2, x3], 1)))
        x5 = self.lrelu(self.conv5(torch.cat([x, x1, x2, x3, x4], 1)))
        return x + x5 * 0.2

In [ ]:
class RRDB(nn.Module):
    def __init__(self, channels, growth_channels=32) -> None:
        super(RRDB, self).__init__()
        self.rdb1 = RRDBBlock(channels, growth_channels)
        self.rdb2 = RRDBBlock(channels, growth_channels)
        self.rdb3 = RRDBBlock(channels, growth_channels)

    def forward(self, x) -> torch.Tensor:
        out = self.rdb1(x)
        out = self.rdb2(out)
        out = self.rdb3(out)
        # Residual scaling ile girişe ekleme
        return x + out * 0.2

In [ ]:
class ESRGANGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_features=64, num_blocks=23, growth_channels=32) -> None:
        super(ESRGANGenerator, self).__init__()
        self.conv_first = nn.Conv2d(in_channels, num_features, kernel_size=3, stride=1, padding=1)
        
        rrdb_blocks = []
        for _ in range(num_blocks):
            rrdb_blocks.append(RRDB(num_features, growth_channels))
        self.RRDB_trunk = nn.Sequential(*rrdb_blocks)
        
        self.trunk_conv = nn.Conv2d(num_features, num_features, kernel_size=3, stride=1, padding=1)
        
        self.upsample = nn.Sequential(
            nn.Conv2d(num_features, num_features * 4, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(num_features, num_features * 4, kernel_size=3, stride=1, padding=1),
            nn.PixelShuffle(2),
            nn.LeakyReLU(0.2, inplace=True)
        )
        self.conv_last = nn.Conv2d(num_features, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        fea = self.conv_first(x)
        trunk = self.trunk_conv(self.RRDB_trunk(fea))
        fea = fea + trunk  # Skip connection
        out = self.upsample(fea)
        out = self.conv_last(out)
        return out

In [ ]:
class ESRGANDiscriminator(nn.Module):
    def __init__(self, in_channels=3, base_channels=64) -> None:
        super(ESRGANDiscriminator, self).__init__()
        layers = []
        layers.append(nn.Conv2d(in_channels, base_channels, kernel_size=3, stride=1, padding=1))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        curr_channels = base_channels
        # Birkaç katmanla özellik çıkarımı (stride kullanarak boyut azaltma)
        for i in range(1, 4):
            layers.append(nn.Conv2d(curr_channels, curr_channels * 2, kernel_size=3, stride=2, padding=1))
            layers.append(nn.BatchNorm2d(curr_channels * 2))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            curr_channels *= 2
        layers.append(nn.Conv2d(curr_channels, curr_channels, kernel_size=3, stride=1, padding=1))
        layers.append(nn.BatchNorm2d(curr_channels))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        self.features = nn.Sequential(*layers)
        # Son katman: global ortalama havuzlama sonrası FC
        self.classifier = nn.Sequential(
            nn.Linear(curr_channels * 16 * 16, 1024),  # Not: Bu değer, giriş görüntü boyutuna bağlıdır.
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 1)
        )

    def forward(self, x) -> torch.Tensor:
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out
